<a href="https://github.com/fgonzaleumbc/DATA601/tree/main/Projects/Project_1/DATA601_Project1" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import numpy as np
import pandas as pd
from PIL import Image
import matplotlib.pyplot as plt
from tqdm import tqdm
import copy
from sklearn.preprocessing import LabelEncoder
import pickle
from sklearn.pipeline import Pipeline

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models

if torch.cuda.is_available():
    print('CUDA is available. Working on GPU')
    DEVICE = torch.device('cuda')
else:
    print('CUDA is not available. Working on CPU')
    DEVICE = torch.device('cpu')

CUDA is available. Working on GPU


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
train_img_paths = pd.read_csv('drive/MyDrive/MURA-v1.1/train_image_paths.csv', header = None, names = ['image_path'])
test_img_paths = pd.read_csv('drive/MyDrive/MURA-v1.1/valid_image_paths.csv', header = None, names = ['image_path'])

In [4]:
def split_img_paths(df):
    temp = df['image_path'].str.split('/', expand=True)
    new_df = pd.DataFrame(df['image_path'])
    new_df['patient_id'] = temp[3]
    new_df['study'] = temp[4].str.split('_', expand = True)[0].str[-1:].astype(int)
    new_df['region'] = temp[2].str.split('_', expand = True)[1]
    map = {'positive':'abnormal', 'negative':'normal'}
    new_df['target'] = temp[4].str.split('_', expand = True)[1]
    new_df['target'] = new_df['target'].map(map)
    #add google drive location to image path
    new_df['image_path'] = 'drive/MyDrive/' + new_df['image_path']
    
    return new_df

In [5]:
mura_train = split_img_paths(train_img_paths)
mura_test = split_img_paths(test_img_paths)

In [6]:
data_transforms = transforms.Compose([
    #
    #note that output channel is 1, will need to change and re-run for use with 3 channels
    #
    transforms.Grayscale(num_output_channels=3),
    transforms.Resize((224,224)),
    transforms.ToTensor()
])

In [7]:
def class_dict(two_targets):
    classes_2 = ['normal', 'abnormal']
    dict_2 = {'normal' : 0, 'abnormal': 1}
    
    return classes_2, dict_2

In [8]:
class MuraDataset(Dataset):
    # change two targets to False if using 14 target data
    def __init__(self, mura_df, two_targets, transform=data_transforms):
        self.mura = mura_df
        self.transform = transform
        self.classes, self.class_to_idx = class_dict(two_targets=two_targets)

    def __len__(self):
        return len(self.mura)

    # retrives image from path in df file and target from df
    def get_image_class(self, index):
        image_path = self.mura.iloc[index]['image_path']
        class_name = self.mura.iloc[index]['target']
        return Image.open(image_path), class_name  

    def __getitem__(self, idx):
        image, class_name = self.get_image_class(idx)
        label = self.class_to_idx[class_name]

        return self.transform(image), label

In [9]:
def training(model, model_name, num_epochs, train_dataloader, test_dataloader):

    loss_function = torch.nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.0003)
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=3, gamma=0.33)

    train_loss_array = []
    train_acc_array = []
    test_loss_array = []
    test_acc_array = []
    lowest_test_loss = np.inf
    best_model = None

    for epoch in tqdm(range(num_epochs)):

        print('Epoch: {} | Learning rate: {}'.format(epoch + 1, scheduler.get_lr()))

        for phase in ['train', 'val']:

            epoch_loss = 0
            epoch_correct_items = 0
            epoch_items = 0

            if phase == 'train':
                model.train()
                with torch.enable_grad():
                    for samples, targets in train_dataloader:
                        samples = samples.to(DEVICE)
                        targets = targets.to(DEVICE)

                        optimizer.zero_grad()
                        outputs = model(samples)
                        loss = loss_function(outputs, targets)
                        preds = outputs.argmax(dim=1)
                        correct_items = (preds == targets).float().sum()
                        
                        loss.backward()
                        optimizer.step()

                        epoch_loss += loss.item()
                        epoch_correct_items += correct_items.item()
                        epoch_items += len(targets)

                train_loss_array.append(epoch_loss / epoch_items)
                train_acc_array.append(epoch_correct_items / epoch_items)

                scheduler.step()

            elif phase == 'val':
                model.eval()
                with torch.no_grad():
                    for samples, targets in test_dataloader:
                        samples = samples.to(DEVICE)
                        targets = targets.to(DEVICE)

                        outputs = model(samples)
                        loss = loss_function(outputs, targets)
                        preds = outputs.argmax(dim=1)
                        correct_items = (preds == targets).float().sum()

                        epoch_loss += loss.item()
                        epoch_correct_items += correct_items.item()
                        epoch_items += len(targets)

                test_loss_array.append(epoch_loss / epoch_items)
                test_acc_array.append(epoch_correct_items / epoch_items)

                if epoch_loss / epoch_items < lowest_test_loss:
                    lowest_test_loss = epoch_loss / epoch_items
                    torch.save(model.state_dict(), '{}_weights.pth'.format(model_name))
                    best_model = copy.deepcopy(model)
                    print("\t| New lowest test loss for {}: {}".format(model_name, lowest_test_loss))

    return best_model, train_loss_array, train_acc_array, test_loss_array, test_acc_array

In [10]:
def dataloader(df):
  df_dl= DataLoader(df, batch_size=64, shuffle=True)
  return df_dl

## **FOREARM**

In [11]:
forearm_train = mura_train[mura_train['region']=='FOREARM']
forearm_test = mura_test[mura_test['region']=='FOREARM']

In [12]:
forearm_train_dataset = MuraDataset(mura_df = forearm_train, two_targets=True)
forearm_test_dataset = MuraDataset(mura_df = forearm_test, two_targets=True)

In [13]:
Forearm_train_dataloader = dataloader(forearm_train_dataset)
Forearm_test_dataloader = dataloader(forearm_test_dataset)
num_epochs = 15

In [14]:
model_densenet161 = models.densenet161(pretrained=True)
for param in model_densenet161.parameters():
    param.requires_grad = False
model_densenet161.classifier = torch.nn.Linear(model_densenet161.classifier.in_features, out_features=200)
model_densenet161 = model_densenet161.to(DEVICE)


densenet161_Forearm_training = training(model=model_densenet161,
                                        model_name='DenseNet161',
                                        num_epochs=num_epochs,
                                        train_dataloader=Forearm_train_dataloader,
                                        test_dataloader=Forearm_test_dataloader)

model_densenet161, train_loss_array, train_acc_array, test_loss_array, test_acc_array = densenet161_Forearm_training

forearm_min_loss = min(test_loss_array)
forearm_min_loss_epoch = test_loss_array.index(forearm_min_loss)
forearm_min_loss_accuracy = test_acc_array[forearm_min_loss_epoch]

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=DenseNet161_Weights.IMAGENET1K_V1`. You can also use `weights=DenseNet161_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/densenet161-8d451a50.pth" to /root/.cache/torch/hub/checkpoints/densenet161-8d451a50.pth
100%|██████████| 110M/110M [00:07<00:00, 16.2MB/s]
  0%|          | 0/15 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/torch/optim/lr_scheduler.py:389: UserWarning: To get the last learning rate computed by the scheduler, 

Epoch: 1 | Learning rate: [0.0003]


  7%|▋         | 1/15 [23:51<5:34:03, 1431.68s/it]

	| New lowest test loss for DenseNet161: 0.014790998146779514
Epoch: 2 | Learning rate: [0.0003]


 13%|█▎        | 2/15 [24:25<2:12:05, 609.69s/it] 

...

 87%|████████▋ | 13/15 [30:47<01:21, 40.88s/it]

	| New lowest test loss for DenseNet161: 0.01035094241367226
Epoch: 14 | Learning rate: [3.557763e-06]


 93%|█████████▎| 14/15 [31:22<00:39, 39.15s/it]

Epoch: 15 | Learning rate: [3.557763e-06]


100%|██████████| 15/15 [31:57<00:00, 127.83s/it]


In [15]:
print("\nTraining results:")
print("\tMin test loss {:.4f} was achieved during epoch #{}".format(forearm_min_loss, forearm_min_loss_epoch + 1))
print("\t test accuracy during min test loss is {:.4f}".format(forearm_min_loss_accuracy))


Training results:
	Min test loss 0.0104 was achieved during epoch #13
	 test accuracy during min test loss is 0.6478


In [ ]:

all_preds = []
all_files = []

model_densenet161.eval()
with torch.no_grad():
    for samples, f_names in tqdm(Forearm_test_dataloader):
        samples = samples.to(DEVICE)
        outputs = model_densenet161(samples)
        preds = outputs.argmax(dim=1)
        all_preds.extend(preds.tolist())
        all_files.extend(f_names)

all_filenames= forearm_test['image_path'].values.tolist()

def class_dict_inverse(two_targets):
    if two_targets==0:
      return 'normal'
    else:
      return 'abnormal'

all_preds_decoded = [class_dict_inverse(i) for i in all_preds]

submission_forearm_df = pd.DataFrame(list(zip(all_filenames, all_preds_decoded)), columns =['File', 'Prediction'])
submission_forearm_df.to_csv('test_predictions_forearm.csv', header=False, index=False) 
submission_forearm_df.head(10)

100%|██████████| 5/5 [00:04<00:00,  1.16it/s]


,File,Prediction
0,drive/MyDrive/MURA-v1.1/valid/XR_FOREARM/patie...,normal
1,drive/MyDrive/MURA-v1.1/valid/XR_FOREARM/patie...,normal
2,drive/MyDrive/MURA-v1.1/valid/XR_FOREARM/patie...,abnormal
3,drive/MyDrive/MURA-v1.1/valid/XR_FOREARM/patie...,normal
4,drive/MyDrive/MURA-v1.1/valid/XR_FOREARM/patie...,normal
5,drive/MyDrive/MURA-v1.1/valid/XR_FOREARM/patie...,normal
6,drive/MyDrive/MURA-v1.1/valid/XR_FOREARM/patie...,abnormal
7,drive/MyDrive/MURA-v1.1/valid/XR_FOREARM/patie...,normal
8,drive/MyDrive/MURA-v1.1/valid/XR_FOREARM/patie...,normal
9,drive/MyDrive/MURA-v1.1/valid/XR_FOREARM/patie...,abnormal


## **HAND**

In [ ]:
hand_train = mura_train[mura_train['region']=='HAND']
hand_test = mura_test[mura_test['region']=='HAND']

In [ ]:
hand_train_dataset = MuraDataset(mura_df = hand_train, two_targets=True)
hand_test_dataset = MuraDataset(mura_df = hand_test, two_targets=True)

In [ ]:
hand_train_dataloader = dataloader(hand_train_dataset)
hand_test_dataloader = dataloader(hand_test_dataset)
num_epochs = 15

In [ ]:
model_densenet161 = models.densenet161(pretrained=True)
for param in model_densenet161.parameters():
    param.requires_grad = False
model_densenet161.classifier = torch.nn.Linear(model_densenet161.classifier.in_features, out_features=200)
model_densenet161 = model_densenet161.to(DEVICE)


densenet161_hand_training = training(model=model_densenet161,
                                        model_name='DenseNet161',
                                        num_epochs=num_epochs,
                                        train_dataloader=hand_train_dataloader,
                                        test_dataloader=hand_test_dataloader)

model_densenet161, train_loss_array, train_acc_array, test_loss_array, test_acc_array = densenet161_hand_training

hand_min_loss = min(test_loss_array)
hand_min_loss_epoch = test_loss_array.index(hand_min_loss)
hand_min_loss_accuracy = test_acc_array[hand_min_loss_epoch]

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=DenseNet161_Weights.IMAGENET1K_V1`. You can also use `weights=DenseNet161_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/densenet161-8d451a50.pth" to /root/.cache/torch/hub/checkpoints/densenet161-8d451a50.pth
100%|██████████| 110M/110M [00:01<00:00, 70.0MB/s]
  0%|          | 0/15 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/torch/optim/lr_scheduler.py:389: UserWarning: To get the last learning rate computed by the scheduler, 

Epoch: 1 | Learning rate: [0.0003]


  7%|▋         | 1/15 [36:48<8:35:12, 2208.02s/it]

	| New lowest test loss for DenseNet161: 0.011789735503818679
Epoch: 2 | Learning rate: [0.0003]


 13%|█▎        | 2/15 [38:25<3:29:21, 966.28s/it] 

...

Epoch: 11 | Learning rate: [1.0781099999999999e-05]


 73%|███████▎  | 11/15 [52:50<07:33, 113.26s/it]

	| New lowest test loss for DenseNet161: 0.010274919090063676
Epoch: 12 | Learning rate: [1.0781099999999999e-05]


 80%|████████  | 12/15 [54:22<05:20, 106.92s/it]

Epoch: 13 | Learning rate: [1.17406179e-06]


In [ ]:
print("\nTraining results:")
print("\tMin test loss {:.4f} was achieved during epoch #{}".format(hand_min_loss, hand_min_loss_epoch + 1))
print("\t test accuracy during min test loss is {:.4f}".format(hand_min_loss_accuracy))


Training results:
	Min test loss 0.0103 was achieved during epoch #14
	 test accuracy during min test loss is 0.6630


## **HUMERUS**

In [ ]:
humerus_train = mura_train[mura_train['region']=='HUMERUS']
humerus_test = mura_test[mura_test['region']=='HUMERUS']

In [ ]:
humerus_train_dataset = MuraDataset(mura_df = humerus_train, two_targets=True)
humerus_test_dataset = MuraDataset(mura_df = humerus_test, two_targets=True)

In [ ]:
Humerus_train_dataloader = dataloader(humerus_train_dataset)
Humerus_test_dataloader = dataloader(humerus_test_dataset)
num_epochs = 15

In [ ]:
model_densenet161 = models.densenet161(pretrained=True)
for param in model_densenet161.parameters():
    param.requires_grad = False
model_densenet161.classifier = torch.nn.Linear(model_densenet161.classifier.in_features, out_features=200)
model_densenet161 = model_densenet161.to(DEVICE)


densenet161_humerus_training = training(model=model_densenet161,
                                        model_name='DenseNet161',
                                        num_epochs=num_epochs,
                                        train_dataloader=Humerus_train_dataloader,
                                        test_dataloader=Humerus_test_dataloader)

model_densenet161, train_loss_array, train_acc_array, test_loss_array, test_acc_array = densenet161_humerus_training

humerus_min_loss = min(test_loss_array)
humerus_min_loss_epoch = test_loss_array.index(humerus_min_loss)
humerus_min_loss_accuracy = test_acc_array[humerus_min_loss_epoch]

  0%|          | 0/15 [00:00<?, ?it/s]

Epoch: 1 | Learning rate: [0.0003]


  7%|▋         | 1/15 [10:48<2:31:22, 648.76s/it]

	| New lowest test loss for DenseNet161: 0.022352507130967245
Epoch: 2 | Learning rate: [0.0003]


 13%|█▎        | 2/15 [11:13<1:00:58, 281.42s/it]

...

 87%|████████▋ | 13/15 [15:46<00:54, 27.47s/it]

Epoch: 14 | Learning rate: [3.557763e-06]


 93%|█████████▎| 14/15 [16:11<00:26, 26.78s/it]

	| New lowest test loss for DenseNet161: 0.010639710765745904
Epoch: 15 | Learning rate: [3.557763e-06]


100%|██████████| 15/15 [16:36<00:00, 66.40s/it]


In [ ]:
print("\nTraining results:")
print("\tMin test loss {:.4f} was achieved during epoch #{}".format(humerus_min_loss, humerus_min_loss_epoch + 1))
print("\t test accuracy during min test loss is {:.4f}".format(humerus_min_loss_accuracy))


Training results:
	Min test loss 0.0106 was achieved during epoch #14
	 test accuracy during min test loss is 0.7326


# **ELBOW**

In [ ]:
elbow_train = mura_train[mura_train['region']=='ELBOW']
elbow_test = mura_test[mura_test['region']=='ELBOW']

In [ ]:
elbow_train_dataset = MuraDataset(mura_df = elbow_train, two_targets=True)
elbow_test_dataset = MuraDataset(mura_df = elbow_test, two_targets=True)

In [ ]:
Elbow_train_dataloader = dataloader(elbow_train_dataset)
Elbow_test_dataloader = dataloader(elbow_test_dataset)
num_epochs = 15

In [ ]:
model_densenet161 = models.densenet161(pretrained=True)
for param in model_densenet161.parameters():
    param.requires_grad = False
model_densenet161.classifier = torch.nn.Linear(model_densenet161.classifier.in_features, out_features=200)
model_densenet161 = model_densenet161.to(DEVICE)


densenet161_elbow_training = training(model=model_densenet161,
                                        model_name='DenseNet161',
                                        num_epochs=num_epochs,
                                        train_dataloader=Elbow_train_dataloader,
                                        test_dataloader=Elbow_test_dataloader)

model_densenet161, train_loss_array, train_acc_array, test_loss_array, test_acc_array = densenet161_elbow_training

elbow_min_loss = min(test_loss_array)
elbow_min_loss_epoch = test_loss_array.index(elbow_min_loss)
elbow_min_loss_accuracy = test_acc_array[elbow_min_loss_epoch]

  0%|          | 0/15 [00:00<?, ?it/s]

Epoch: 1 | Learning rate: [0.0003]


  7%|▋         | 1/15 [31:20<7:18:47, 1880.52s/it]

	| New lowest test loss for DenseNet161: 0.011664818307404876
Epoch: 2 | Learning rate: [0.0003]


 13%|█▎        | 2/15 [32:44<2:58:27, 823.65s/it] 

...

 87%|████████▋ | 13/15 [48:11<03:02, 91.11s/it]

	| New lowest test loss for DenseNet161: 0.00900949348685562
Epoch: 14 | Learning rate: [3.557763e-06]


 93%|█████████▎| 14/15 [49:35<01:28, 88.83s/it]

Epoch: 15 | Learning rate: [3.557763e-06]


100%|██████████| 15/15 [50:59<00:00, 203.97s/it]


In [ ]:
print("\nTraining results:")
print("\tMin test loss {:.4f} was achieved during epoch #{}".format(elbow_min_loss, elbow_min_loss_epoch + 1))
print("\t test accuracy during min test loss is {:.4f}".format(elbow_min_loss_accuracy))


Training results:
	Min test loss 0.0090 was achieved during epoch #13
	 test accuracy during min test loss is 0.7333


## **SHOULDER**

In [ ]:
shoulder_train = mura_train[mura_train['region']=='SHOULDER']
shoulder_test = mura_test[mura_test['region']=='SHOULDER']


In [ ]:
shoulder_train_dataset = MuraDataset(mura_df = shoulder_train, two_targets=True)
shoulder_test_dataset = MuraDataset(mura_df = shoulder_test, two_targets=True)

In [ ]:
Shoulder_train_dataloader = dataloader(shoulder_train_dataset)
Shoulder_test_dataloader = dataloader(shoulder_test_dataset)
num_epochs = 15

In [ ]:
model_densenet161 = models.densenet161(pretrained=True)
for param in model_densenet161.parameters():
    param.requires_grad = False
model_densenet161.classifier = torch.nn.Linear(model_densenet161.classifier.in_features, out_features=200)
model_densenet161 = model_densenet161.to(DEVICE)


densenet161_shoulder_training = training(model=model_densenet161,
                                        model_name='DenseNet161',
                                        num_epochs=num_epochs,
                                        train_dataloader=Shoulder_train_dataloader,
                                        test_dataloader=Shoulder_test_dataloader)

model_densenet161, train_loss_array, train_acc_array, test_loss_array, test_acc_array = densenet161_shoulder_training

shoulder_min_loss = min(test_loss_array)
shoulder_min_loss_epoch = test_loss_array.index(shoulder_min_loss)
shoulder_min_loss_accuracy = test_acc_array[shoulder_min_loss_epoch]

  0%|          | 0/15 [00:00<?, ?it/s]

Epoch: 1 | Learning rate: [0.0003]


  7%|▋         | 1/15 [03:34<50:02, 214.46s/it]

	| New lowest test loss for DenseNet161: 0.010544008831783044
Epoch: 2 | Learning rate: [0.0003]


 13%|█▎        | 2/15 [05:51<36:32, 168.65s/it]

...

	| New lowest test loss for DenseNet161: 0.008948914269994461
Epoch: 14 | Learning rate: [3.557763e-06]


 93%|█████████▎| 14/15 [33:27<02:18, 138.15s/it]

Epoch: 15 | Learning rate: [3.557763e-06]


100%|██████████| 15/15 [35:45<00:00, 143.00s/it]

	| New lowest test loss for DenseNet161: 0.008914005481116937


In [ ]:
print("\nTraining results:")
print("\tMin test loss {:.4f} was achieved during epoch #{}".format(shoulder_min_loss, shoulder_min_loss_epoch + 1))
print("\t test accuracy during min test loss is {:.4f}".format(shoulder_min_loss_accuracy))


Training results:
	Min test loss 0.0089 was achieved during epoch #15
	 test accuracy during min test loss is 0.7336


## **FINGER**

In [ ]:
finger_train = mura_train[mura_train['region']=='FINGER']
finger_test = mura_test[mura_test['region']=='FINGER']


In [ ]:
finger_train_dataset = MuraDataset(mura_df = finger_train, two_targets=True)
finger_test_dataset = MuraDataset(mura_df = finger_test, two_targets=True)

In [ ]:
Finger_train_dataloader = dataloader(finger_train_dataset)
Finger_test_dataloader = dataloader(finger_test_dataset)
num_epochs = 15

In [ ]:
model_densenet161 = models.densenet161(pretrained=True)
for param in model_densenet161.parameters():
    param.requires_grad = False
model_densenet161.classifier = torch.nn.Linear(model_densenet161.classifier.in_features, out_features=200)
model_densenet161 = model_densenet161.to(DEVICE)


densenet161_finger_training = training(model=model_densenet161,
                                        model_name='DenseNet161',
                                        num_epochs=num_epochs,
                                        train_dataloader=Finger_train_dataloader,
                                        test_dataloader=Finger_test_dataloader)

model_densenet161, train_loss_array, train_acc_array, test_loss_array, test_acc_array = densenet161_finger_training

finger_min_loss = min(test_loss_array)
finger_min_loss_epoch = test_loss_array.index(finger_min_loss)
finger_min_loss_accuracy = test_acc_array[finger_min_loss_epoch]

  0%|          | 0/15 [00:00<?, ?it/s]

Epoch: 1 | Learning rate: [0.0003]


  7%|▋         | 1/15 [35:26<8:16:06, 2126.21s/it]

	| New lowest test loss for DenseNet161: 0.011907374781279657
Epoch: 2 | Learning rate: [0.0003]


 13%|█▎        | 2/15 [36:50<3:20:22, 924.82s/it] 

...

 87%|████████▋ | 13/15 [51:55<03:01, 90.55s/it]

Epoch: 14 | Learning rate: [3.557763e-06]


 93%|█████████▎| 14/15 [53:18<01:28, 88.23s/it]

	| New lowest test loss for DenseNet161: 0.010030600745352126
Epoch: 15 | Learning rate: [3.557763e-06]


100%|██████████| 15/15 [54:40<00:00, 218.73s/it]


In [ ]:
print("\nTraining results:")
print("\tMin test loss {:.4f} was achieved during epoch #{}".format(finger_min_loss, finger_min_loss_epoch + 1))
print("\t test accuracy during min test loss is {:.4f}".format(finger_min_loss_accuracy))


Training results:
	Min test loss 0.0100 was achieved during epoch #14
	 test accuracy during min test loss is 0.6985


## **WRIST**

In [16]:
wrist_train = mura_train[mura_train['region']=='WRIST']
wrist_test = mura_test[mura_test['region']=='WRIST']

In [17]:
wrist_train_dataset = MuraDataset(mura_df = wrist_train, two_targets=True)
wrist_test_dataset = MuraDataset(mura_df = wrist_test, two_targets=True)

In [18]:
Wrist_train_dataloader = DataLoader(wrist_train_dataset, batch_size=64, shuffle=True)
Wrist_test_dataloader = DataLoader(wrist_test_dataset, batch_size=64, shuffle=False)
num_epochs = 15

In [19]:
model_densenet161 = models.densenet161(pretrained=True)
for param in model_densenet161.parameters():
    param.requires_grad = False
model_densenet161.classifier = torch.nn.Linear(model_densenet161.classifier.in_features, out_features=200)
model_densenet161 = model_densenet161.to(DEVICE)

densenet161_Wrist_training = training(model=model_densenet161,
                                        model_name='DenseNet161',
                                        num_epochs=num_epochs,
                                        train_dataloader=Wrist_train_dataloader,
                                        test_dataloader=Wrist_test_dataloader)

model_densenet161, train_loss_array, train_acc_array, test_loss_array, test_acc_array = densenet161_Wrist_training

wrist_min_loss = min(test_loss_array)
wrist_min_loss_epoch = test_loss_array.index(wrist_min_loss)
wrist_min_loss_accuracy = test_acc_array[wrist_min_loss_epoch]

  0%|          | 0/15 [00:00<?, ?it/s]

Epoch: 1 | Learning rate: [0.0003]


  7%|▋         | 1/15 [2:03:08<28:43:53, 7388.13s/it]

	| New lowest test loss for DenseNet161: 0.009633857717282493
Epoch: 2 | Learning rate: [0.0003]


 13%|█▎        | 2/15 [2:05:51<11:19:54, 3138.04s/it]

...

 87%|████████▋ | 13/15 [2:36:47<06:39, 199.68s/it]

	| New lowest test loss for DenseNet161: 0.008007795636136543
Epoch: 14 | Learning rate: [3.557763e-06]


 93%|█████████▎| 14/15 [2:39:36<03:10, 190.24s/it]

	| New lowest test loss for DenseNet161: 0.008005113630627645
Epoch: 15 | Learning rate: [3.557763e-06]


100%|██████████| 15/15 [2:42:24<00:00, 649.65s/it]


In [20]:
print("\nTraining results:")
print("\tMin test loss {:.4f} was achieved during epoch #{}".format(wrist_min_loss, wrist_min_loss_epoch + 1))
print("\t test accuracy during min test loss is {:.4f}".format(wrist_min_loss_accuracy))


Training results:
	Min test loss 0.0080 was achieved during epoch #14
	 test accuracy during min test loss is 0.7724
